In [2]:
## AUDIOBOOK BUSINESS CASE STUDY

In [3]:
#EXTRACTING THE DATA FROM THE CSV

In [4]:
import numpy as np
from sklearn import preprocessing

raw_csv_data = np.loadtxt('Audiobooks_data.csv', delimiter = ',')
unscaled_inputs_all = raw_csv_data [:,1:-1]
targets_all = raw_csv_data[:,-1]

In [5]:
# SHUFFLING THE DATA

In [6]:
shuffled_indices = np.arange(unscaled_inputs_all.shape[0])
np.random.shuffle(shuffled_indices)

# Use the shuffled indices to shuffle the inputs and targets.
unscaled_inputs_all = unscaled_inputs_all[shuffled_indices]
targets_all = targets_all[shuffled_indices]

In [7]:
# BALANCING THE DATASET

In [8]:
num_one_targets = int(np.sum(targets_all))
zero_targets_counter = 0
indices_to_remove = []

for i in range (targets_all.shape[0]):
    if targets_all[i] == 0:
        zero_targets_counter += 1
        if zero_targets_counter > num_one_targets:
            indices_to_remove.append(i)

unscaled_inputs_equal_priors = np.delete(unscaled_inputs_all, indices_to_remove, axis=0)
targets_equal_priors = np.delete(targets_all, indices_to_remove, axis=0)

In [9]:
# STANDARDIZING INPUTS

In [10]:
scaled_inputs = preprocessing.scale(unscaled_inputs_equal_priors)

In [11]:
# SHUFFLING THE DATA

In [12]:
shuffled_indices = np.arange(scaled_inputs.shape[0])
np.random.shuffle(shuffled_indices)

shuffled_inputs = scaled_inputs[shuffled_indices]
shuffled_targets = targets_equal_priors[shuffled_indices] 

In [13]:
# SPLITTING THE DATA INTO TRAIN, VALIDATION AND TEST

In [14]:
samples_count = shuffled_inputs.shape[0]

train_samples_counts = int(0.8*samples_count)
validation_samples_count = int(0.1*samples_count)
test_samples_count = samples_count - train_samples_counts - validation_samples_count

train_inputs = shuffled_inputs[:train_samples_counts]
train_targets = shuffled_targets[:train_samples_counts]

validation_inputs = shuffled_inputs[train_samples_counts:train_samples_counts+validation_samples_count]
validation_targets = shuffled_targets[train_samples_counts:train_samples_counts+validation_samples_count]

test_inputs = shuffled_inputs[train_samples_counts+validation_samples_count:]
test_targets =shuffled_targets[train_samples_counts+validation_samples_count:]

print(np.sum(train_targets), train_samples_counts, np.sum(train_targets)/train_samples_counts)
print(np.sum(validation_targets), validation_samples_count, np.sum(validation_targets)/validation_samples_count)
print(np.sum(test_targets), test_samples_count, np.sum(test_targets)/test_samples_count)

1782.0 3579 0.4979044425817267
221.0 447 0.49440715883668906
234.0 448 0.5223214285714286


In [15]:
# SAVING THE THREE DATASETS IN *.NPZ

In [16]:
np.savez('Audiobooks_data_train', inputs = train_inputs, targets = train_targets)
np.savez('Audiobooks_data_validation', inputs = validation_inputs, targets = validation_targets)
np.savez('Audiobooks_data_test', inputs = test_inputs, targets = test_targets)

In [17]:
# IMPORT THE RELEVANT LIBRARIES

In [18]:
import numpy as np
import tensorflow as tf

In [19]:
# LOADING THE NOZ FORMATTED DATA

In [20]:
npz = np.load('Audiobooks_data_train.npz')

train_inputs = npz['inputs'].astype(float)
train_targets = npz['targets'].astype(int)

npz = np.load('Audiobooks_data_validation.npz')
validation_inputs, validation_targets = npz['inputs'].astype(float), npz['targets'].astype(int)

npz = np.load('Audiobooks_data_test.npz')
test_inputs, test_targets = npz['inputs'].astype(float), npz['targets'].astype(int)

In [21]:
## MODEL
# OUTLINE, OPTIMIZERS, LOSS, EARLY STOPPING AND TRANING

In [22]:
input_size = 10
output_size = 2
hidden_layer_size = 50

model = tf.keras.Sequential([
    tf.keras.layers.Dense(hidden_layer_size,activation='relu'),
    tf.keras.layers.Dense(hidden_layer_size,activation='relu'),
    tf.keras.layers.Dense(output_size,activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy']) 

batch_size = 100

max_epochs = 100

early_stopping = tf.keras.callbacks.EarlyStopping(patience=2)

model.fit(train_inputs,
          train_targets,
          batch_size = batch_size,
          epochs = max_epochs,
          callbacks = [early_stopping],
          validation_data = (validation_inputs, validation_targets),
          verbose = 2)

Epoch 1/100
36/36 - 1s - loss: 0.6392 - accuracy: 0.6547 - val_loss: 0.5397 - val_accuracy: 0.7584 - 513ms/epoch - 14ms/step
Epoch 2/100
36/36 - 0s - loss: 0.4896 - accuracy: 0.7625 - val_loss: 0.4558 - val_accuracy: 0.7785 - 59ms/epoch - 2ms/step
Epoch 3/100
36/36 - 0s - loss: 0.4384 - accuracy: 0.7695 - val_loss: 0.4190 - val_accuracy: 0.8054 - 68ms/epoch - 2ms/step
Epoch 4/100
36/36 - 0s - loss: 0.4135 - accuracy: 0.7879 - val_loss: 0.3927 - val_accuracy: 0.8054 - 66ms/epoch - 2ms/step
Epoch 5/100
36/36 - 0s - loss: 0.3988 - accuracy: 0.7930 - val_loss: 0.3856 - val_accuracy: 0.8054 - 64ms/epoch - 2ms/step
Epoch 6/100
36/36 - 0s - loss: 0.3883 - accuracy: 0.8022 - val_loss: 0.3737 - val_accuracy: 0.8098 - 63ms/epoch - 2ms/step
Epoch 7/100
36/36 - 0s - loss: 0.3810 - accuracy: 0.8027 - val_loss: 0.3672 - val_accuracy: 0.8076 - 61ms/epoch - 2ms/step
Epoch 8/100
36/36 - 0s - loss: 0.3785 - accuracy: 0.8022 - val_loss: 0.3629 - val_accuracy: 0.8121 - 64ms/epoch - 2ms/step
Epoch 9/100
36

In [23]:
# TESTING THE MODEL

In [24]:
print(test_inputs.shape)

(448, 10)


In [25]:
print(test_targets.shape)

(448,)


In [26]:
test_loss, test_accuracy = model.evaluate(test_inputs, test_targets)

14/14 [==============================] - 0s 997us/step - loss: 0.3444 - accuracy: 0.8214


In [56]:
print('\nTest loss: {0:.2f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy * 100))


Test loss: 0.34. Test accuracy: 82.14%
